In [133]:
import pandas as pd
import bs4 as bs
import requests
import time
import numpy as np

In [134]:
df = pd.read_excel('Fin_Res_Q3_22_23_12_Feb_2023_in_V1.xls')
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
urls = df["res_url"].tolist()
urls

['https://www.moneycontrol.com/financials/arhamtechnologies/results/quarterly-results/AT25#AT25',
 'https://www.moneycontrol.com/financials/maxindialimited/results/consolidated-quarterly-results/MIL19#MIL19',
 'http://www.moneycontrol.com/financials/jbfindustries/results/quarterly-results/JBF',
 'https://www.moneycontrol.com/financials/anandrathiwealth/results/consolidated-quarterly-results/ARW#ARW',
 'http://www.moneycontrol.com/financials/cyient/results/consolidated-quarterly-results/IE07',
 'http://www.moneycontrol.com/financials/dennetworks/results/consolidated-quarterly-results/DN02',
 'http://www.moneycontrol.com/financials/gmbreweries/results/quarterly-results/GMB#GMB',
 'http://www.moneycontrol.com/financials/infosys/results/consolidated-quarterly-results/IT',
 'https://www.moneycontrol.com/financials/abansholdings/results/quarterly-results/AH10#AH10',
 'https://www.moneycontrol.com/financials/alletechnologies/results/quarterly-results/AT26#AT26',
 'https://www.moneycontrol.com

In [135]:
def get_data(url):
    resp = requests.get(url)
    soup = bs.BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'class': 'mctable1'})
    data = []
    if table is None:
        return data
    for row in table.findAll('tr'):
        cols = row.findAll('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return data

In [136]:
outdf = pd.DataFrame(columns=["SYMB", " DEC '22_R", " DEC '22_V", " DEC '22_DEPS",
                                " DEC '22_Interest", " DEC '22_Gross NPA", " DEC '22_Net NPA", "LUU",
                                    "RES_CATG_1", "RES_CATG_2", "L_RES_DT", "DATA_AVAILABLE",
                                    "Last Available Data", "TIME STAMP", "RES_URL", "SECTOR"])

In [137]:
def fill_df(urls):
    for url in urls[:21]:
        data = get_data(url)
        if len(data) == 0:
            outdf.loc[urls.index(url), "RES_URL"] = df.loc[urls.index(url)+1, "res_url"]
            outdf.loc[urls.index(url), "DATA_AVAILABLE"] = "No"
            outdf.loc[urls.index(url), "TIME STAMP"] = time.strftime("%d/%m/%Y %H:%M:%S")
            outdf.loc[urls.index(url), "SECTOR"] = df.loc[urls.index(url)+1, "sectr"]
            outdf.loc[urls.index(url),'SYMB'] = df.loc[urls.index(url)+1, "symb"]
            outdf.loc[urls.index(url), "LUU"] = df.loc[urls.index(url)+1, "LUU"]
            outdf.loc[urls.index(url), "RES_CATG_1"] = df.loc[urls.index(url)+1, "res_catg1"]
            outdf.loc[urls.index(url), "RES_CATG_2"] = df.loc[urls.index(url)+1, "res_catg2"]
            outdf.loc[urls.index(url), "L_RES_DT"] = df.loc[urls.index(url)+1, "l_res_dt"]
        elif len(data) > 0:
            dg = pd.DataFrame(data,index=None,columns=None)
            dg.columns = dg.iloc[0]
            dg = dg.drop(dg.index[0])
            dg = dg.iloc[:, :2]
            outdf.loc[urls.index(url),'SYMB'] = df.loc[urls.index(url)+1, "symb"]
            if df.loc[urls.index(url)+1, "LUU"] == np.nan:
                outdf.loc[urls.index(url), "LUU"] = "NULL"
            else:
                outdf.loc[urls.index(url), "LUU"] = df.loc[urls.index(url)+1, "LUU"]
            outdf.loc[urls.index(url), "RES_CATG_1"] = df.loc[urls.index(url)+1, "res_catg1"]
            outdf.loc[urls.index(url), "RES_CATG_2"] = df.loc[urls.index(url)+1, "res_catg2"]
            outdf.loc[urls.index(url), "L_RES_DT"] = df.loc[urls.index(url)+1, "l_res_dt"]
            outdf.loc[urls.index(url), "RES_URL"] = df.loc[urls.index(url)+1, "res_url"]
            outdf.loc[urls.index(url), "SECTOR"] = df.loc[urls.index(url)+1, "sectr"]
            outdf.loc[urls.index(url), " DEC '22_R"] = dg.loc[1][1]
            outdf.loc[urls.index(url), " DEC '22_V"] = dg.loc[28][1]
            outdf.loc[urls.index(url), " DEC '22_DEPS"] = dg.loc[37][1]
            outdf.loc[urls.index(url), " DEC '22_Interest"] = dg.loc[20][1]
            outdf.loc[urls.index(url), "DATA_AVAILABLE"] = "Yes"
            outdf.loc[urls.index(url), "Last Available Data"] = dg.columns[1]
            outdf.loc[urls.index(url), "TIME STAMP"] = time.strftime("%d/%m/%Y %H:%M:%S")
            if outdf.loc[urls.index(url), "SECTOR"] == "Banks":
                outdf.loc[urls.index(url), " DEC '22_Gross NPA"] = dg.loc[35][1]
                outdf.loc[urls.index(url), " DEC '22_Net NPA"] = dg.loc[36][1]
            else:
                outdf.loc[urls.index(url), " DEC '22_Gross NPA"] = "0"
                outdf.loc[urls.index(url), " DEC '22_Net NPA"] = "0"
    return outdf

In [140]:
output = fill_df(urls)
output = output.fillna("0", inplace=True)
print(output)

None
